In [85]:
import json
import requests
import urllib
import copy

In [7]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display

In [8]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io/legacy'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
        print( child['status'], child['actor']['agent'],nresults )
    return results

In [9]:
def submit_to_devars(m):
    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

def retrieve_devars_results(m):
    return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def submit_to_ciars(m):
    return submit_to_ars(m,ars_url='https://ars.ci.transltr.io/ars/api',arax_url='https://arax.ncats.io')

def retrieve_ciars_results(m):
    return retrieve_ars_results(m,ars_url='https://ars.ci.transltr.io/ars/api')


In [10]:
def printjson(j):
    print(json.dumps(j,indent=4))

def print_json(j):
    printjson(j)

In [11]:
def parse(json):
    keylist = list(json.keys())
    for key in keylist:
        if key in KEYS:
            if (isinstance(json[KEYS.get(key)],dict)):
                parse(json[key])
        elif (isinstance(json[key],dict)):
            parse(json[key])

In [12]:
def name_lookup(text):
    url= f'https://name-resolution-sri.renci.org/lookup?string={text}&offset=0&limit=10'
    response = requests.post(url)
    printjson(response.json())

In [13]:
def one_hop_message(subject_category,object_category,predicate=None,subject_id = None, object_id = None):
    query_graph = {
    "nodes": {
        'a':{
            "category": subject_category
        },
        'b':{
            "category": object_category
        }
    },
    "edges": {
        'ab':{
            "subject": "a",
            "object": "b",
            #"predicate": predicate
        }
    }
    }
    if predicate is not None:
        query_graph['edges']['ab']['predicate'] = predicate
    if subject_id is not None:
        query_graph['nodes']['a']['id'] = subject_id
    if object_id is not None:
        query_graph['nodes']['b']['id'] = object_id
    message = {"message": {"query_graph": query_graph},'knowledge_graph':{"nodes": [], "edges": [],}, 'results':[]}
    return message

In [14]:
def keys_exist(element, *keys):
    if not isinstance(element, dict):
        raise AttributeError('keys_exists() expects dict as first argument.')
    if len(keys) == 0:
        raise AttributeError('keys_exists() expects at least two arguments, one given.')

    _element = element
    for key in keys:
        try:
            _element = _element[key]
            if _element is None:
                return False
        except KeyError:
            return False
    return True

def get_safe(element,*keys):
    '''
    :param element: JSON to be processed
    :param keys: list of keys in order to be traversed. e.g. "fields","data","message","results
    :return: the value of the terminal key if present or None if not
    '''
    if element is None:
        return None
    _element = element
    for key in keys:
        try:
            _element = _element[key]
            if _element is None:
                return None
            if key == keys[-1]:
                return _element
        except KeyError:
            return None
    return None

In [15]:
'''
This pulls approved drugs by pulling the MAX_FDA_APPROVAL_PHASE string from the description of the node
This is by no means a complete solution and I should touch base with Vlado about the implementation of 
FDA approval data from MolePro for a more complete solution.
'''

def get_approved(results):
    approved = []
    for r in results:
        nodes = get_safe(result[r],"message","knowledge_graph","nodes")
        if nodes is not None:
            for node in nodes:
                attributes = get_safe(nodes[node],"attributes")
                if attributes is not None:
                    for attribute in attributes:
                        if("original_attribute_name" in attribute):
                            if(attribute["original_attribute_name"]=="description"):
                                if("MAX_FDA_APPROVAL_PHASE: 4" in attribute["value"]):
                                    approved.append(node)
    return approved

In [16]:
def synonymize(ids, url="https://nodenormalization-sri.renci.org/1.1/get_normalized_nodes"):
        if( isinstance(ids,str) ):
            ids = [ids]
        params = {"curie":ids}
        url_curies = urllib.parse.urlencode(params, True)
        syn_url=url+"?"+url_curies
        syn_response=requests.get(syn_url)
        synonyms= syn_response.json()
        return synonyms
        
                    

In [17]:
def create_synonym_table(messages):
    synonyms={}
    for message in messages.values():
        nodes = get_safe(message,"message","knowledge_graph","nodes")
        if nodes is not None:
            sym = synonymize(nodes)
            if(len(synonyms)==0):
                synonyms= sym
            else:
                absent = []
                for node in nodes:
                    if(node in synonyms.items()):
                        continue
                    else:
                        absent.append(node)
                absent_syms = synonymize(absent)
                synonyms.update(absent_syms)
    return synonyms
                


In [18]:
def create_synonym_tables(messages):
    synonyms={}
    for entry in messages:
        actor = entry
        message = messages[actor]
        nodes = get_safe(message,"message","knowledge_graph","nodes")
        if nodes is not None:
            sym = synonymize(nodes)
            synonyms[actor]=sym
    return synonyms
                

In [79]:
def canonical_id_lists(synonym_tables):
    pks ={}
    for entry in synonym_tables:
        pks[entry]=[]
        for k,v in synonym_tables[entry].items():
            if v is None:
                pks[entry].append(k)
            else:
                canonical_id=v["id"]["identifier"]
                if (canonical_id not in pks[entry]):
                    pks[entry].append(canonical_id)
    return pks

In [118]:
def get_consensus_nodes(canonical_id_lists):
    consensus={}
    canon = copy.deepcopy(canonical_id_lists)
    my_keys= canon.keys()
    my_values=canon.values()
    for actor, id_list in canonical_id_lists.items():
        for actor2, id_list2 in canon.items():
            #Surely, there is a better way to do this with slices or something
            if(actor==actor2):
                continue
            else:
                s1 = set(id_list)
                s2 = set(id_list2)
                inter = (s1 & s2)
                if (inter):
                    for node in inter:
                        if node in consensus.keys():
                            consensus[node].add(actor)
                            consensus[node].add(actor2)
                        else:
                            consensus[node]={actor,actor2}      
        canon.pop(actor,None)
    return consensus
        

# KCNMA1

In [20]:
kcq={
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "ids": [
                        "HGNC:6284"
                    ],
                    "categories": [
                        "biolink:Gene"
                    ]
                },
                "n1": {
		             "categories":[
                         "biolink:ChemicalSubstance"
                        ]
                }
            },
            "edges": {
                "e0": {
                    "subject": "n0",
                    "object": "n1"

                }
            }
        }
    }
}
  


In [21]:
kcresult = submit_to_devars(kcq)

https://arax.ncats.io/?source=ARS&id=94f6b020-9a30-45c2-9a32-75eb9e8dff9a


In [23]:
result = retrieve_devars_results(kcresult)

Done
Done
Done ara-aragorn 871
Done
Done ara-arax 100
Done
Done ara-bte 181
Done
Done ara-unsecret 152
Done
Done kp-genetics 0
Done
Done kp-molecular 99
Done
Done ara-explanatory 0
Done
Done ara-improving 5
Done
Done kp-cam 0
Done
Done kp-textmining 0
Done
Done kp-openpredict 0
Done
ARS Error kp-cohd 0
Error
Error kp-chp 0
Done
Done kp-icees 0
Done
Done kp-icees-dili 0


In [83]:
syns = create_synonym_tables(result)

In [119]:
canon = canonical_id_lists(syns)
con = get_consensus_nodes(canon)       
print (len(con))

227


In [117]:
for k in sorted(con,key=lambda k: len(con[k]),reverse=True):
    print(k, con[k])

PUBCHEM.COMPOUND:5413 {'ara-arax', 'ara-unsecret', 'ara-aragorn', 'ara-bte', 'kp-molecular'}
PUBCHEM.COMPOUND:5560 {'ara-arax', 'ara-unsecret', 'ara-aragorn', 'ara-bte', 'kp-molecular'}
PUBCHEM.COMPOUND:9951995 {'ara-arax', 'ara-unsecret', 'ara-aragorn', 'ara-bte', 'kp-molecular'}
PUBCHEM.COMPOUND:4552 {'ara-arax', 'ara-unsecret', 'ara-aragorn', 'ara-bte', 'kp-molecular'}
PUBCHEM.COMPOUND:4189 {'ara-arax', 'ara-unsecret', 'ara-aragorn', 'ara-bte', 'kp-molecular'}
PUBCHEM.COMPOUND:3226 {'ara-arax', 'ara-unsecret', 'ara-aragorn', 'ara-bte', 'kp-molecular'}
PUBCHEM.COMPOUND:3639 {'ara-arax', 'ara-unsecret', 'ara-aragorn', 'ara-bte', 'kp-molecular'}
PUBCHEM.COMPOUND:3019 {'ara-arax', 'ara-unsecret', 'ara-aragorn', 'ara-bte', 'kp-molecular'}
PUBCHEM.COMPOUND:2315 {'ara-arax', 'ara-unsecret', 'ara-aragorn', 'ara-bte', 'kp-molecular'}
PUBCHEM.COMPOUND:3647 {'ara-arax', 'ara-unsecret', 'ara-aragorn', 'ara-bte', 'kp-molecular'}
PUBCHEM.COMPOUND:3562 {'ara-arax', 'ara-unsecret', 'ara-aragorn', '

In [120]:
approved_list = get_approved(result)

['CHEMBL.COMPOUND:CHEMBL1015',
 'CHEMBL.COMPOUND:CHEMBL104',
 'CHEMBL.COMPOUND:CHEMBL1054',
 'CHEMBL.COMPOUND:CHEMBL112570',
 'CHEMBL.COMPOUND:CHEMBL1131',
 'CHEMBL.COMPOUND:CHEMBL1201039',
 'CHEMBL.COMPOUND:CHEMBL1201201',
 'CHEMBL.COMPOUND:CHEMBL1255',
 'CHEMBL.COMPOUND:CHEMBL1257',
 'CHEMBL.COMPOUND:CHEMBL136478',
 'CHEMBL.COMPOUND:CHEMBL1371',
 'CHEMBL.COMPOUND:CHEMBL1421',
 'CHEMBL.COMPOUND:CHEMBL1616046',
 'CHEMBL.COMPOUND:CHEMBL1684',
 'CHEMBL.COMPOUND:CHEMBL1763',
 'CHEMBL.COMPOUND:CHEMBL181',
 'CHEMBL.COMPOUND:CHEMBL2108350',
 'CHEMBL.COMPOUND:CHEMBL2146121',
 'CHEMBL.COMPOUND:CHEMBL313972',
 'CHEMBL.COMPOUND:CHEMBL333826',
 'CHEMBL.COMPOUND:CHEMBL389621',
 'CHEMBL.COMPOUND:CHEMBL424',
 'CHEMBL.COMPOUND:CHEMBL435',
 'CHEMBL.COMPOUND:CHEMBL545',
 'CHEMBL.COMPOUND:CHEMBL569',
 'CHEMBL.COMPOUND:CHEMBL71',
 'CHEMBL.COMPOUND:CHEMBL71595',
 'CHEMBL.COMPOUND:CHEMBL720',
 'CHEMBL.COMPOUND:CHEMBL74',
 'CHEMBL.COMPOUND:CHEMBL91',
 'CHEMBL.COMPOUND:CHEMBL931',
 'CHEMBL.COMPOUND:CHEMBL959